# Bike Rental Data Management Project

## Importing libraries and data

In [36]:
import os
import datetime
import string

import pandas as pd
import numpy as np

import sqlalchemy

Next, we will be loading the monthly datasets for Citi Bike and concatenate them in a single `pandas` dataframe. It is important to note that this information is from 2016. I will try to run a similar analysis for 2022 data eventually.

In [37]:
df_bike = pd.DataFrame()

In [38]:
# use for-loop to create unified dataframe through df_temp
for f in sorted(os.listdir('datafiles/citibike_and_weather_data_2016')):
    if f.startswith('JC'):
        df_temp = pd.read_csv('datafiles/citibike_and_weather_data_2016/' + f)
        df_bike = pd.concat([df_bike, df_temp], axis=0)

# reset index after concatenation
df_bike.reset_index(drop=True, inplace=True)

# add an 'id' column for counts and future primary key usage
df_bike['id'] = df_bike.index

Inspect both the beginning and end of our dataframe:

In [39]:
df_bike.head(5)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,id
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2,0
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1,3
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0,4


In [40]:
df_bike.tail(5)

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender,id
247579,557,2016-12-31 23:10:16,2016-12-31 23:19:33,3214,Essex Light Rail,40.712774,-74.036486,3203,Hamilton Park,40.727596,-74.044247,24465,Subscriber,1981.0,2,247579
247580,2749,2016-12-31 23:29:39,2017-01-01 00:15:29,3183,Exchange Place,40.716247,-74.033459,3183,Exchange Place,40.716247,-74.033459,24389,Customer,NaN,0,247580
247581,173,2016-12-31 23:44:37,2016-12-31 23:47:31,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,24641,Subscriber,1978.0,1,247581
247582,2424,2016-12-31 23:44:50,2017-01-01 00:25:14,3214,Essex Light Rail,40.712774,-74.036486,3214,Essex Light Rail,40.712774,-74.036486,26219,Subscriber,1960.0,2,247582
247583,2419,2016-12-31 23:44:50,2017-01-01 00:25:10,3214,Essex Light Rail,40.712774,-74.036486,3214,Essex Light Rail,40.712774,-74.036486,24471,Subscriber,1956.0,1,247583


We can check that there are almost 250k records in this df. There are also some observations to take into account based on the data dictionary that came with the starter kit:
* `Trip Duration` is in seconds
* `Gender` can be `0=unknown`, `1=male`, and `2=female`
* `User Type` has values `Customer= 24 hr pass or 3-day user` and `Subscriber=Annual member`

I will now calculate some basic descriptive statistics for numerical columns from the df:

In [41]:
df_bike.describe()

,Trip Duration,Start Station ID,Start Station Latitude,Start Station Longitude,End Station ID,End Station Latitude,End Station Longitude,Bike ID,Birth Year,Gender,id
count,2.475840e+05,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,247584.000000,228585.000000,247584.000000,247584.000000
mean,8.856305e+02,3207.065206,40.723121,-74.046438,3203.572553,40.722594,-74.045855,24935.260481,1979.335276,1.123534,123791.500000
std,3.593798e+04,26.955103,0.008199,0.011211,61.579494,0.007958,0.011283,748.469712,9.596809,0.518687,71471.488861
min,6.100000e+01,3183.000000,40.692640,-74.096937,147.000000,40.692216,-74.096937,14552.000000,1900.000000,0.000000,0.000000
25%,2.480000e+02,3186.000000,40.717732,-74.050656,3186.000000,40.716540,-74.050444,24491.000000,1974.000000,1.000000,61895.750000
50%,3.900000e+02,3201.000000,40.721525,-74.044247,3199.000000,40.721124,-74.043117,24609.000000,1981.000000,1.000000,123791.500000
75%,6.660000e+02,3211.000000,40.727596,-74.038051,3211.000000,40.727224,-74.036486,24719.000000,1986.000000,1.000000,185687.250000
max,1.632981e+07,3426.000000,40.752559,-74.032108,3426.000000,40.801343,-73.957390,27274.000000,2000.000000,2.000000,247583.000000


Some things to note from the above description:
* There is something interesting going on with the `Trip Durantion`, given it jump five orders of magnitude between the 75th percentile and the max
* The minimum `Birth Year` is 1900, which equates to an age of 116
* The minimum for the `End Station ID` is 147, which is not congruent with the minimum for the `Start Station ID`. This is something that I will need to look into to avoid errors in my analysis
* Latitud and Longitude both show small stdev, so it is safe to assume there are no major discrepancies between these set of values

Moving along, I will now scout for missing values and duplicates:

In [42]:
df_bike.isna().sum()

Trip Duration                  0
Start Time                     0
Stop Time                      0
Start Station ID               0
Start Station Name             0
Start Station Latitude         0
Start Station Longitude        0
End Station ID                 0
End Station Name               0
End Station Latitude           0
End Station Longitude          0
Bike ID                        0
User Type                    380
Birth Year                 18999
Gender                         0
id                             0
dtype: int64

In [43]:
df_bike.duplicated().sum()

0

At first glance, it is clear there is some major problems with the `Birth Year` of some users, as well as the `User Type `in fewer instances. Keep in mind that the `Gender` column has values of `0` which correspondon to an `Unknown` genre, so this is also something that will be addressed further down the road.

For now, let's continue with the exploratory analysis and see what data types are present in the df:

In [44]:
df_bike.dtypes

Trip Duration                int64
Start Time                  object
Stop Time                   object
Start Station ID             int64
Start Station Name          object
Start Station Latitude     float64
Start Station Longitude    float64
End Station ID               int64
End Station Name            object
End Station Latitude       float64
End Station Longitude      float64
Bike ID                      int64
User Type                   object
Birth Year                 float64
Gender                       int64
id                           int64
dtype: object

It is key to note that both `Start Time` and `Stop Time` need to be casted into `datetime64` data type. I will also have to address the problem with `Birth Year` (which should be an integer) once I have corrected all the missing values.

In [45]:
# casting objects as timestamps with pd.to_datetime()
df_bike['Start Time'] = pd.to_datetime(df_bike['Start Time'])
df_bike['Stop Time'] = pd.to_datetime(df_bike['Stop Time'])

df_bike.dtypes

Trip Duration                       int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth Year                        float64
Gender                              int64
id                                  int64
dtype: object

# Cleaning and Transforming Bike Rental Data

Quick summary of our initial data exploration:
* `Birth Year`: suspicious minimum and missing data
* `User Type`: missing data
* `Trip Duration`: suspicious maximum
* `Gender`: what are the unknowns
* `End Station ID`: incongruent minimum

Now we will update columns names to something that will help us in the future once we stablish our db schema:

In [46]:
df_bike.columns = df_bike.columns.map(lambda x: x.replace(' ', '_').lower())
df_bike.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2,0
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1,3
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0,4


## Calculating `age` using `birth_year`

In [47]:
df_bike['age'] = 2016 - df_bike['birth_year'].values
df_bike['age'].sort_values(ascending=False)[:10]

215055    116.0
136211     82.0
164274     79.0
187190     79.0
180373     79.0
213567     79.0
177320     76.0
112336     76.0
149803     76.0
124501     75.0
Name: age, dtype: float64

We can consider the age of 116 to be an outlier for out dataset, so we will remove it since we are not expecting other riders with that age.

In [48]:
df_bike.drop([215055], inplace=True)

Now let's examine the missing data in `birth_year`:

In [49]:
missing_birth_year = df_bike[df_bike[['birth_year']].isnull().any(axis=1)]
missing_birth_year.head()

,trip_duration,start_time,stop_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bike_id,user_type,birth_year,gender,id,age
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0,4,NaN
5,883,2016-01-01 01:03:28,2016-01-01 01:18:11,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24442,Customer,NaN,0,5,NaN
22,988,2016-01-01 03:16:33,2016-01-01 03:33:02,3196,Riverview Park,40.744319,-74.043991,3209,Brunswick St,40.724176,-74.050656,24662,Customer,NaN,0,22,NaN
53,3090,2016-01-01 11:07:15,2016-01-01 11:58:46,3203,Hamilton Park,40.727596,-74.044247,3203,Hamilton Park,40.727596,-74.044247,24444,Customer,NaN,0,53,NaN
57,788,2016-01-01 11:50:30,2016-01-01 12:03:39,3210,Pershing Field,40.742677,-74.051789,3195,Sip Ave,40.730743,-74.063784,24573,Customer,NaN,0,57,NaN


Let's look at the `user_type` and `gender` of this subset of users:

In [50]:
missing_birth_year['user_type'].value_counts()

user_type
Customer      15470
Subscriber     3529
Name: count, dtype: int64

In [51]:
missing_birth_year['gender'].value_counts()

gender
0    18999
Name: count, dtype: int64

From these results, we can notice that:
* all records with missing `birth_year` also have unknown `gender`
* customer `user_type` account for the vast majority of the missing `birth_year` data.

Let's take a look at the percentage of `user_type` with missing `birth_year` data:

In [52]:
missing_birth_year['user_type'].value_counts() / df_bike['user_type'].value_counts()

user_type
Customer      0.996714
Subscriber    0.015232
Name: count, dtype: float64

In [53]:
missing_birth_year[missing_birth_year['user_type'] == 'Subscriber']['gender'].value_counts() / df_bike[df_bike['user_type'] == 'Subscriber']['gender'].value_counts()

gender
0    0.796434
1         NaN
2         NaN
Name: count, dtype: float64

In [35]:
missing_birth_year[missing_birth_year['user_type'] == 'Customer']['gender'].value_counts() / df_bike[df_bike['user_type'] == 'Customer']['gender'].value_counts()

gender
0    1.0
1    NaN
2    NaN
Name: count, dtype: float64

Notice that:
* 99.7% of customers are missing `birth_year`, most likely because that information is optional in the rental contract
* 1.5% of subscribers are missing `birth_year`, this must be the case because to become an annual member, they need to provide more information to the rental company
* all records with missing `birth_year` also have unknown `gender`.